<a href="https://www.kaggle.com/code/melaneemelanee/war-and-peace-with-rnn?scriptVersionId=101607339" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

This notebook is taken from: (by Milad Vazan)

https://www.researchgate.net/publication/360017232_yadgyry_myq_az_aswl_awlyh_ta_sakht_shbkhhay_sby_myq_ba_paytwnPage

Page: 155



In [1]:
training_file = '../input/war-and-peace/warpeace_input.txt'
raw_text = open(training_file, 'r').read()
raw_text = raw_text.lower()
raw_text[:100]

'"well, prince, so genoa and lucca are now just family estates of the\nbuonapartes. but i warn you, if'

In [2]:
n_chars = len(raw_text)
print('Total charachter: {}' .format(n_chars))


Total charachter: 3196213


In [3]:
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print('Total vocabulary(unic charachter): {}'.format(n_vocab))
print(chars)

Total vocabulary(unic charachter): 56
['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'ä', 'é', 'ê']


In [4]:
index_to_char = dict((i, c) for i, c in enumerate(chars))
char_to_index = dict((i, c) for i, c in enumerate(chars))
print(char_to_index)

{0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: '*', 8: ',', 9: '-', 10: '.', 11: '/', 12: '0', 13: '1', 14: '2', 15: '3', 16: '4', 17: '5', 18: '6', 19: '7', 20: '8', 21: '9', 22: ':', 23: ';', 24: '=', 25: '?', 26: 'a', 27: 'b', 28: 'c', 29: 'd', 30: 'e', 31: 'f', 32: 'g', 33: 'h', 34: 'i', 35: 'j', 36: 'k', 37: 'l', 38: 'm', 39: 'n', 40: 'o', 41: 'p', 42: 'q', 43: 'r', 44: 's', 45: 't', 46: 'u', 47: 'v', 48: 'w', 49: 'x', 50: 'y', 51: 'z', 52: 'à', 53: 'ä', 54: 'é', 55: 'ê'}


In [5]:
import numpy as np
seq_length = 100
n_seq = int(n_chars / seq_length)

In [6]:
X= np.zeros((n_seq, seq_length, n_vocab))
Y= np.zeros((n_seq, seq_length, n_vocab))


In [7]:
#for i in range(n_seq):
    #x_sequence = raw_text[i*seq_length: (i+1)*seq_length]
    #x_sequence_ohe=np.zeros((seq_length, n_vocab))
    #for j in range(seq_length):
        #char = x_sequence[j]
        #index=char_to_index[char]
        #x_sequence_ohe[j][index]=1.
    #X[i]=x_sequence_ohe
    #y_sequence=raw_text[i*seq_length+1:(i+1)*seq_length+1]
    #y_sequence_ohe=np.zeros((seq_length, n_vocab))
    #for j in range(seq_length):
        #char=y_sequence[j]
        #index=char_to_index[char]
        #y_sequence_ohe[j][index]=1.
    #Y[i]=y_sequence_ohe     

In [8]:
#

In [9]:
X.shape

(31962, 100, 56)

In [10]:
Y.shape

(31962, 100, 56)

In [11]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras import optimizers
from tensorflow import keras

In [12]:
batch_size = 100
n_layer=2
hidden_units=800
n_epoch=300
dropout=0.3

In [13]:
model = Sequential()
model.add(SimpleRNN(hidden_units, activation='relu', input_shape=(None, n_vocab), return_sequences=True))
model.add(Dropout(dropout))
for i in range(n_layer - 1):
    model.add(SimpleRNN(hidden_units, activation='relu', return_sequences=True))
    model.add(Dropout(dropout))
model.add(TimeDistributed(Dense(n_vocab)))
model.add(Activation('softmax'))

2022-07-24 04:21:10.319062: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [14]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, None, 800)         685600    
_________________________________________________________________
dropout (Dropout)            (None, None, 800)         0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 800)         1280800   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 800)         0         
_________________________________________________________________
time_distributed (TimeDistri (None, None, 56)          44856     
_________________________________________________________________
activation (Activation)      (None, None, 56)          0         
Total params: 2,011,256
Trainable params: 2,011,256
Non-trainable params: 0
______________________________________________

In [15]:
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping


In [16]:
early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=50, verbose=1, mode='min')

In [17]:

def generate_text(model, gen_length, n_vocab, index_to_char):
    """
    Generating text using the RNN model
    @param model: current RNN model
    @param gen_length: number of characters we want to generate
    @param n_vocab: number of unique characters
    @param index_to_char: index to character mapping
    @return:
    """
    # Start with a randomly picked character
    index = np.random.randint(n_vocab)
    y_char = [index_to_char[index]]
    X = np.zeros((1, gen_length, n_vocab))
    for i in range(gen_length):
        X[0, i, index] = 1.
        indices = np.argmax(model.predict(X[:, max(0, i - 99):i + 1, :])[0], 1)
        index = indices[-1]
        y_char.append(index_to_char[index])
    return ('').join(y_char)

In [18]:
class ResultChecker(Callback):
    def __init__(self, model, N, gen_length):
        self.model = model
        self.N = N
        self.gen_length = gen_length

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.N == 0:
            result = generate_text(self.model, self.gen_length, n_vocab, index_to_char)
            print('\nMy War and Peace:\n' + result)

In [19]:

filepath="weights/weights_layer_%d_hidden_%d_epoch_{epoch:03d}_loss_{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [20]:
#model.fit(X, Y, batch_size=batch_size, verbose=1, epochs=n_epoch,
                 #callbacks=[ResultChecker(model, 2, 200), checkpoint, early_stop])